In [2]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from io import StringIO
import pygwalker as pyg
import os
from tqdm import tqdm
import polars.selectors as cs
from scipy.stats import zscore

reading competition editions json

In [3]:
ce_json_path = './skillcorner-competition-editions.json'
ce_df1 = pl.read_json(ce_json_path)
ce_df1.head(1)
#ce_df.columns

count,next,previous,results
i64,null,null,list[struct[4]]
938,null,null,"[{0,{0,""FIFA"",""World Cup"",""male"",""adult""},{10,2014,2014,""2014""},""FIFA - World Cup - 2014""}, {1,{1,""ENG"",""Premier League"",""male"",""adult""},{2,2015,2016,""2015/2016""},""ENG - Premier League - 2015/2016""}, … {1129,{221,""INT"",""Pinatar National Team Friendlies"",""male"",""U21""},{128,2025,2025,""2025""},""INT - Pinatar National Team Friendlies - 2025""}]"


In [4]:
#ce_df2 = ce_df1.explode('results')
ce_df2 = ce_df1.select('results').explode('results')
ce_df2 = ce_df2.unnest('results')
ce_df2.head(1)

id,competition,season,name
i64,struct[5],struct[4],str
0,"{0,""FIFA"",""World Cup"",""male"",""adult""}","{10,2014,2014,""2014""}","""FIFA - World Cup - 2014"""


In [5]:
ce_name_id = ce_df2.select([
    pl.col('name'),
    pl.col('id'),
    ]
)
ce_name_id.head(1)

name,id
str,i64
"""FIFA - World Cup - 2014""",0


In [6]:
ce_competition = ce_df2.select('competition').unnest('competition')
ce_competition = ce_competition.rename({col: f"competition_{col}" for col in ce_competition.columns})
ce_competition.head(1)

competition_id,competition_area,competition_name,competition_gender,competition_age_group
i64,str,str,str,str
0,"""FIFA""","""World Cup""","""male""","""adult"""


In [7]:
ce_season = ce_df2.select('season').unnest('season')
ce_season = ce_season.rename({col: f"season_{col}" for col in ce_season.columns})
ce_season.head(1)

season_id,season_start_year,season_end_year,season_name
i64,i64,i64,str
10,2014,2014,"""2014"""


In [8]:
ce = pl.concat([
    ce_name_id,
    ce_season,
    ce_competition
],
how = "horizontal",)
ce.head(10)

name,id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""FRA - Ligue 1 - 2016/2017""",5,3,2016,2017,"""2016/2017""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""FRA - Ligue 1 - 2017/2018""",6,4,2017,2018,"""2017/2018""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""ESP - LaLiga - 2016/2017""",7,3,2016,2017,"""2016/2017""",4,"""ESP""","""LaLiga""","""male""","""adult"""
"""ESP - LaLiga - 2017/2018""",8,4,2017,2018,"""2017/2018""",4,"""ESP""","""LaLiga""","""male""","""adult"""


In [9]:
# checking if columns contain more than 1 value
#ce.select("competition_age_group").unique()

reading json player file- test on 1 file


In [10]:
position_json_testpath = './skillcorner-20250214/metrics-ce249-rAM.json'

position_test_df = pl.read_json(position_json_testpath)

In [11]:
position_test_df.columns
#position_test_df.head(10)

['Count Sprint OTIP 1',
 'Count Sprint TIP 2',
 'date',
 'Count High Deceleration OTIP 2',
 'Count Medium Acceleration 1',
 'Count Medium Deceleration TIP',
 'Count High Deceleration OTIP 1',
 'Minutes TIP 1',
 'season_name',
 'Distance',
 'Count Sprint TIP 1',
 'Distance OTIP',
 'Minutes TIP 2',
 'match_name',
 'Count Medium Deceleration TIP 1',
 'Count Medium Acceleration 2',
 'Sprinting Distance OTIP 2',
 'Sprinting Distance TIP',
 'Count Sprint 2',
 'Count High Deceleration TIP 1',
 'Minutes OTIP 2',
 'Running Distance OTIP 2',
 'Distance TIP 2',
 'Count HSR 1',
 'match_id',
 'Count Sprint',
 'Count High Acceleration 1',
 'Count Medium Acceleration TIP',
 'Minutes 2',
 'Count Sprint 1',
 'Distance TIP 1',
 'Distance OTIP 2',
 'Count High Deceleration OTIP',
 'Sprinting Distance',
 'Count HSR 2',
 'Count Medium Deceleration 1',
 'HSR Distance OTIP 1',
 'Minutes TIP',
 'competition_edition_id',
 'Count Medium Acceleration OTIP 1',
 'player_name',
 'Count High Acceleration TIP 2',
 'H

In [12]:
#define the path to the entire directory
json_dir_path = './skillcorner-20250214/'

### Look at correlations between parameters
Looking at the correlations across all competitions across all positions to see if we can leave out certain parameters

In [13]:
json_list = os.listdir(json_dir_path)
#json_list

In [14]:
read_json_list = []

for jsonf in tqdm(json_list):
    json_path = f"{json_dir_path}{jsonf}"
    json_df = pl.read_json(json_path)

    read_json_list.append(json_df)

corr_df = pl.concat(read_json_list)

  0%|          | 0/1652 [00:00<?, ?it/s]

100%|██████████| 1652/1652 [00:27<00:00, 60.29it/s] 


In [15]:
corr_df.shape

(449537, 117)

In [16]:
corr_df.head()

Count Sprint OTIP 1,Count Sprint TIP 2,date,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Minutes TIP 1,season_name,Distance,Count Sprint TIP 1,Distance OTIP,Minutes TIP 2,match_name,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Minutes OTIP 2,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Minutes 2,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,…,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,team_name,quality_check,Sprinting Distance 2,Minutes,competition_id,Count HSR TIP 2,Distance 2,PSV-99,Count HSR TIP,Minutes 1,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,position,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Minutes OTIP 1,team,Count Medium Deceleration OTIP 1
i64,i64,str,i64,i64,i64,i64,f64,str,i64,i64,i64,f64,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,i64,f64,i64,i64,i64,f64,i64,f64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,f64,str,i64
1,1,"""2021-07-25""",1,52,17,3,13.13,"""2021/2022""",13026,1,5727,11.08,"""KV Mechelen v Royal Antwerp FC""",8,48,18,30,2,2,17.3,662,1558,33,44999,5,3,22,49.48,3,1874,2744,4,148,26,42,225,…,10,92,12,8701,358,8,11,3432,292,2983,100,89,"""Royal Antwerp FC""",true,56,96.63,16,6,6216,25.7,19,47.15,9,3,34,47,"""AM""",1,661,0,17,745,387,47,18.27,"""Royal Antwerp FC""",17
1,3,"""2021-07-24""",0,36,20,1,13.2,"""2021/2022""",9969,1,3534,12.32,"""Oud-Heverlee Leuven v SV Zulte…",9,41,0,136,5,1,15.07,224,1775,20,45002,8,3,25,50.07,3,1660,1739,1,202,19,31,104,…,14,48,11,10932,275,5,17,3435,343,1795,77,77,"""Oud-Heverlee Leuven""",true,154,96.19,16,11,5127,28.3,20,46.12,11,3,17,46,"""AM""",2,610,1,7,476,201,32,14.52,"""Oud-Heverlee Leuven""",11
1,0,"""2021-07-24""",1,39,33,2,14.52,"""2021/2022""",8709,2,2876,7.93,"""Oud-Heverlee Leuven v SV Zulte…",22,25,7,26,1,1,6.57,172,1278,21,45002,5,1,26,24.95,4,2289,938,3,105,13,57,89,…,8,95,18,14625,129,7,26,3567,297,1938,64,82,"""SV Zulte-Waregem""",true,10,71.07,16,5,3089,26.4,18,46.12,11,4,14,25,"""AM""",2,724,2,7,421,292,23,13.2,"""SV Zulte-Waregem""",13
1,4,"""2021-07-24""",3,37,18,2,13.2,"""2021/2022""",8976,4,3351,7.15,"""Oud-Heverlee Leuven v SV Zulte…",13,27,20,132,6,2,8.48,235,1242,25,45002,12,0,21,28.03,6,1883,1326,5,207,25,44,114,…,8,76,13,9835,310,9,44,3125,276,2025,64,70,"""Oud-Heverlee Leuven""",true,131,74.15,16,12,3566,27.4,25,46.12,5,6,20,26,"""AM""",1,603,0,9,594,284,26,14.52,"""Oud-Heverlee Leuven""",13
1,0,"""2021-07-25""",1,47,23,2,15.68,"""2021/2022""",11626,2,4902,12.98,"""Club Brugge KV v KAS Eupen""",13,39,4,36,1,1,19.18,320,1397,23,44998,5,2,29,60.17,4,2340,2252,3,60,12,44,112,…,11,56,18,11595,81,6,36,3737,142,2650,86,85,"""KAS Eupen""",true,4,110.42,16,4,5369,25.5,17,50.25,10,2,17,41,"""AM""",0,603,1,8,334,253,47,18.05,"""KAS Eupen""",18


In [17]:
pl.Config.set_tbl_cols(150)
corr_df.describe()

statistic,Count Sprint OTIP 1,Count Sprint TIP 2,date,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Minutes TIP 1,season_name,Distance,Count Sprint TIP 1,Distance OTIP,Minutes TIP 2,match_name,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Minutes OTIP 2,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Minutes 2,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,Minutes TIP,competition_edition_id,Count Medium Acceleration OTIP 1,player_name,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,group,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Minutes OTIP,player_birthdate,short_name,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,team_name,quality_check,Sprinting Distance 2,Minutes,competition_id,Count HSR TIP 2,Distance 2,PSV-99,Count HSR TIP,Minutes 1,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,position,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Minutes OTIP 1,team,Count Medium Deceleration OTIP 1
str,f64,f64,str,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
"""count""",449429.0,449511.0,"""449537""",448984.0,449429.0,449537.0,449319.0,449537.0,"""449537""",449537.0,449429.0,449537.0,449537.0,"""449537""",449429.0,449511.0,449511.0,449537.0,449511.0,449299.0,449537.0,449511.0,449511.0,449429.0,449537.0,449537.0,449429.0,449537.0,449537.0,449429.0,449429.0,449511.0,449537.0,449537.0,449511.0,449429.0,449429.0,449537.0,449537.0,449429.0,"""449537""",448747.0,449537.0,449429.0,449429.0,449537.0,449537.0,449537.0,449537.0,449429.0,449537.0,"""449537""",449537.0,449537.0,449511.0,449537.0,449429.0,448984.0,449319.0,449429.0,449299.0,449537.0,449537.0,449429.0,449429.0,448747.0,449537.0,"""449324""","""449537""",449537.0,449537.0,449511.0,449511.0,449511.0,449511.0,449511.0,449429.0,449511.0,449537.0,449537.0,449511.0,449429.0,449429.0,449537.0,449511.0,449429.0,449429.0,449537.0,449511.0,449429.0,449537.0,449537.0,"""449537""",449537.0,449511.0,449537.0,449537.0,449511.0,449511.0,449537.0,449537.0,449537.0,449511.0,449511.0,449537.0,449511.0,"""449537""",449511.0,449537.0,449537.0,449511.0,449537.0,449429.0,449537.0,449537.0,"""449537""",449429.0
"""null_count""",108.0,26.0,"""0""",553.0,108.0,0.0,218.0,0.0,"""0""",0.0,108.0,0.0,0.0,"

In [71]:
corr_df.select("quality_check").unique()

quality_check
bool
true


#### Investigating null values

In [19]:
null_count_df = corr_df.null_count()
null_count_df

Count Sprint OTIP 1,Count Sprint TIP 2,date,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Minutes TIP 1,season_name,Distance,Count Sprint TIP 1,Distance OTIP,Minutes TIP 2,match_name,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Minutes OTIP 2,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Minutes 2,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,Minutes TIP,competition_edition_id,Count Medium Acceleration OTIP 1,player_name,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,group,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Minutes OTIP,player_birthdate,short_name,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,team_name,quality_check,Sprinting Distance 2,Minutes,competition_id,Count HSR TIP 2,Distance 2,PSV-99,Count HSR TIP,Minutes 1,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,position,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Minutes OTIP 1,team,Count Medium Deceleration OTIP 1
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
108,26,0,553,108,0,218,0,0,0,108,0,0,0,108,26,26,0,26,238,0,26,26,108,0,0,108,0,0,108,108,26,0,0,26,108,108,0,0,108,0,790,0,108,108,0,0,0,0,108,0,0,0,0,26,0,108,553,218,108,238,0,0,108,108,790,0,213,0,0,0,26,26,26,26,26,108,26,0,0,26,108,108,0,26,108,108,0,26,108,0,0,0,0,26,0,0,26,26,0,0,0,26,26,0,26,0,26,0,0,26,0,108,0,0,0,108


In [20]:
corr_df_missing = (corr_df.filter(pl.any_horizontal(pl.all().is_null())))
corr_df_missing.shape

(1824, 117)

In [21]:
corr_df_missing

Count Sprint OTIP 1,Count Sprint TIP 2,date,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Minutes TIP 1,season_name,Distance,Count Sprint TIP 1,Distance OTIP,Minutes TIP 2,match_name,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Minutes OTIP 2,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Minutes 2,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,Minutes TIP,competition_edition_id,Count Medium Acceleration OTIP 1,player_name,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,group,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Minutes OTIP,player_birthdate,short_name,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,team_name,quality_check,Sprinting Distance 2,Minutes,competition_id,Count HSR TIP 2,Distance 2,PSV-99,Count HSR TIP,Minutes 1,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,position,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Minutes OTIP 1,team,Count Medium Deceleration OTIP 1
i64,i64,str,i64,i64,i64,i64,f64,str,i64,i64,i64,f64,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,i64,f64,i64,i64,i64,f64,i64,f64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,f64,str,i64
0,3,"""2022-02-02""",1,9,24,null,5.0,"""2021/2022""",7379,0,1006,23.07,"""KV Kortrijk v Royal Antwerp FC""",2,65,14,54,5,1,8.47,189,2795,7,92757,7,3,44,61.48,2,682,1006,1,137,26,14,0,28.07,249,0,"""Jesaja Herrmann""",1,83,78,997,15,1,19,8,148,1,"""Midfield""",8,1153,1,5,0,0,null,0,0,3,189,173,5,2,9.75,"""2000-02-08""","""J. Herrmann""",33,298,83,8,54,980,220,0,12,14,3,37,60,7,60031,355,4,0,3477,658,0,74,65,"""KV Kortrijk""",true,77,69.43,16,17,6382,26.7,22,7.95,22,11,8,51,"""AM""",2,806,0,8,433,78,12,1.28,"""KV Kortrijk""",0
1,0,"""2021-07-30""",1,47,8,2,11.58,"""2021/2022""",6615,4,2605,1.92,"""KRC Genk v KV Oostende""",8,17,24,76,2,2,6.25,89,0,24,45284,8,4,18,15.02,6,1677,688,3,144,10,36,57,13.5,249,10,"""Makhtar Gueye""",null,131,248,5050,10,3,29,8,371,1,"""Center Forward""",15,916,2,6,225,1,1,6,1,2,314,666,15,null,22.32,"""1997-12-04""","""M. Gueye""",34,248,74,4,0,250,0,30,5,54,4,0,108,18,12287,137,6,76,1677,0,1917,64,49,"""KV Oostende""",true,36,61.17,16,0,1565,29.0,15,46.15,0,4,11,13,"""CF""",2,371,2,5,459,322,15,16.07,"""KV Oostende""",11
2,0,"""2021-07-25""",3,41,9,4,13.13,"""2021/2022""",7785,0,3545,1.95,"""KV Mechelen v Royal Antwerp FC""",9,21,69,0,2,0,7.27,255,0,30,44999,5,1,

In [22]:
corr_df_missing.describe()

statistic,Count Sprint OTIP 1,Count Sprint TIP 2,date,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Minutes TIP 1,season_name,Distance,Count Sprint TIP 1,Distance OTIP,Minutes TIP 2,match_name,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Minutes OTIP 2,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Minutes 2,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,Minutes TIP,competition_edition_id,Count Medium Acceleration OTIP 1,player_name,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,group,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Minutes OTIP,player_birthdate,short_name,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,team_name,quality_check,Sprinting Distance 2,Minutes,competition_id,Count HSR TIP 2,Distance 2,PSV-99,Count HSR TIP,Minutes 1,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,position,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Minutes OTIP 1,team,Count Medium Deceleration OTIP 1
str,f64,f64,str,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
"""count""",1716.0,1798.0,"""1824""",1271.0,1716.0,1824.0,1606.0,1824.0,"""1824""",1824.0,1716.0,1824.0,1824.0,"""1824""",1716.0,1798.0,1798.0,1824.0,1798.0,1586.0,1824.0,1798.0,1798.0,1716.0,1824.0,1824.0,1716.0,1824.0,1824.0,1716.0,1716.0,1798.0,1824.0,1824.0,1798.0,1716.0,1716.0,1824.0,1824.0,1716.0,"""1824""",1034.0,1824.0,1716.0,1716.0,1824.0,1824.0,1824.0,1824.0,1716.0,1824.0,"""1824""",1824.0,1824.0,1798.0,1824.0,1716.0,1271.0,1606.0,1716.0,1586.0,1824.0,1824.0,1716.0,1716.0,1034.0,1824.0,"""1611""","""1824""",1824.0,1824.0,1798.0,1798.0,1798.0,1798.0,1798.0,1716.0,1798.0,1824.0,1824.0,1798.0,1716.0,1716.0,1824.0,1798.0,1716.0,1716.0,1824.0,1798.0,1716.0,1824.0,1824.0,"""1824""",1824.0,1798.0,1824.0,1824.0,1798.0,1798.0,1824.0,1824.0,1824.0,1798.0,1798.0,1824.0,1798.0,"""1824""",1798.0,1824.0,1824.0,1798.0,1824.0,1716.0,1824.0,1824.0,"""1824""",1716.0
"""null_count""",108.0,26.0,"""0""",553.0,108.0,0.0,218.0,0.0,"""0""",0.0,108.0,0.0,0.0,"""0""",108.0,26.0,26.0,0.0,26.0,238.0,0.0,26.0,26.0,108.0,0.0,0.0,108.0,0.0,0.0,108.0,108.0,26.0,0.0,0.0,26.0,108.0,108.0,0.0,0.0,108.0,"""0""",790.0,0.0,108.0,108.0,0.0,0.0,0.0,0.0,108.0,0.0,"""0""",0.0,0.0,26.0,0.0,108.0,553.0,218.0

In [23]:
#removing the rows containing null values
corr_df_clean = corr_df.drop_nulls()
corr_df_clean.shape

# deleted 1824 rows

(447713, 117)

In [24]:
corr_df_clean.describe()
#moeilijk de vinger op te leggen waardoor 0 waarden waren binnen de kolommen.
#lijkt wel dat er geen minute koloomen meer zijn met waarden gelijk aan 0


statistic,Count Sprint OTIP 1,Count Sprint TIP 2,date,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Minutes TIP 1,season_name,Distance,Count Sprint TIP 1,Distance OTIP,Minutes TIP 2,match_name,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Minutes OTIP 2,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Minutes 2,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,Minutes TIP,competition_edition_id,Count Medium Acceleration OTIP 1,player_name,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,group,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Minutes OTIP,player_birthdate,short_name,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,team_name,quality_check,Sprinting Distance 2,Minutes,competition_id,Count HSR TIP 2,Distance 2,PSV-99,Count HSR TIP,Minutes 1,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,position,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Minutes OTIP 1,team,Count Medium Deceleration OTIP 1
str,f64,f64,str,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
"""count""",447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""","""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,"""447713""",447713.0
"""null_count""",0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,"""0""",0.0,

In [25]:
# chechk if all null values are gone
corr_df_missing = (corr_df_clean.filter(pl.any_horizontal(pl.all().is_null())))
corr_df_missing.shape

(0, 117)

#### Calculating correlations

In [26]:
#filter out the numerical columns of the dataframe to calculate correlations
num_corr_df = corr_df_clean.select(
    pl.col(pl.Int64),
    pl.col(pl.Float64)
    )
num_corr_df.describe()

statistic,Count Sprint OTIP 1,Count Sprint TIP 2,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Distance,Count Sprint TIP 1,Distance OTIP,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,competition_edition_id,Count Medium Acceleration OTIP 1,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,Sprinting Distance 2,competition_id,Count HSR TIP 2,Distance 2,Count HSR TIP,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Count Medium Deceleration OTIP 1,Minutes TIP 1,Minutes TIP 2,Minutes OTIP 2,Minutes 2,Minutes TIP,Minutes OTIP,Minutes,PSV-99,Minutes 1,Minutes OTIP 1
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0,447713.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [27]:
#filtering on outliers
num_corr_df_np = num_corr_df.to_numpy()
zscores = zscore(num_corr_df_np)

In [28]:
print(zscores)

[[-0.67875168 -0.2785834  -0.68535577 ... -1.48827036 -0.20383203
   1.06939253]
 [-0.67875168  0.92678417 -1.50754065 ... -0.07587984 -0.51961674
  -0.08582592]
 [-0.67875168 -0.88126718 -0.68535577 ... -1.10801137 -0.51961674
  -0.49246281]
 ...
 [ 0.00308546  0.32410039  0.959014   ... -0.56478425 -0.7771499
   0.58574107]
 [ 1.36675976  0.32410039 -0.68535577 ...  0.52166999  0.14874312
  -0.52326863]
 [ 1.36675976  0.92678417  0.13682911 ... -0.02155713  0.14874312
   0.24071583]]


In [29]:
# calculating Z-scores
zscore_array = (np.abs(zscore(num_corr_df.to_numpy())) >= 3).all(axis=1)

In [30]:
# checking if there are any extreme outliers
# there are no outliers that are more than 3 standard deviations away from the variable mean 
outlier_number = 0

for boolean in tqdm(zscore_array):
    if boolean == True:
        outlier_number += 1
    else:
        continue

print(outlier_number)

100%|██████████| 447713/447713 [00:00<00:00, 549299.29it/s]

0


In [31]:
#initialize empty dataframe to store correlations
correlations_df = pl.DataFrame(schema=[
    ("var1", pl.Utf8),
    ("var2", pl.Utf8),
    ("corr", pl.Float64)
])

In [33]:
correlations = num_corr_df.corr()

columns = correlations.columns

for i in range(len(columns)):
    for j in range(i + 1, len(columns)):  # Only upper triangle to avoid repetition
        correlation = correlations[i, j]

        if correlation >0.8 or correlation < -0.8:
            correlations_df.extend(pl.DataFrame({"var1":[columns[i]], "var2":[columns[j]], "corr":[correlations[i, j]]}))
            #print(f"Correlation between {columns[i]} and {columns[j]}: {correlations[i, j]}")
        else:
            continue

In [34]:
correlations.shape

(106, 106)

In [35]:
correlations.head()

Count Sprint OTIP 1,Count Sprint TIP 2,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Distance,Count Sprint TIP 1,Distance OTIP,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,competition_edition_id,Count Medium Acceleration OTIP 1,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,Sprinting Distance 2,competition_id,Count HSR TIP 2,Distance 2,Count HSR TIP,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Count Medium Deceleration OTIP 1,Minutes TIP 1,Minutes TIP 2,Minutes OTIP 2,Minutes 2,Minutes TIP,Minutes OTIP,Minutes,PSV-99,Minutes 1,Minutes OTIP 1
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,0.167814,0.155291,0.135876,-0.037555,0.30048,0.089854,0.213646,0.130729,-0.050901,0.063284,0.232951,0.225075,0.267848,0.084984,0.082457,-0.00669,0.349229,0.009152,0.581995,0.19769,0.006888,0.681179,-0.019678,0.046118,0.293963,0.520217,0.192615,0.090229,0.534871,0.006389,0.175383,0.066716,0.446228,0.137273,0.172362,0.289926,0.80944,-0.085505,0.04186,0.020304,0.089639,0.092383,0.138414,0.252648,0.204089,0.1768,0.120688,0.211395,0.433804,0.079198,0.109653,0.151918,0.148405,0.117851,0.089561,0.32827,0.146715,0.18992,0.040732,0.173791,0.087916,0.12282,0.791012,0.085508,0.655079,0.223826,0.009134,0.592486,0.0024,-0.003137,0.202514,0.29343,0.211858,-0.015291,0.036875,0.185388,0.115843,0.07963,0.259164,-0.083753,0.103048,0.022406,0.124882,-0.012929,0.174431,0.380967,0.044317,0.132127,0.033119,0.217982,0.184465,0.377313,0.422222,0.146818,0.118009,-0.04902,-0.025477,0.018492,-0.019225,-0.04318,0.069655,-0.006023,0.357823,0.049705,0.110476
0.167814,1.0,0.246872,0.099392,0.18695,0.226415,0.216383,0.48623,0.027286,0.067682,0.201475,0.198629,0.751632,0.789696,0.322483,0.051704,0.25318,0.387473,-0.008979,0.72112,0.299377,0.388093,0.450871,0.12555,0.065928,0.293226,0.662856,0.542191,0.087671,0.090531,-0.012504,-0.126668,0.402931,0.142537,0.462017,0.095536,0.471643,0.243482,-0.056405,0.007518,0.22728,0.435928,-0.0045,0.209997,0.219772,0.410895,-0.017289,0

In [89]:
pl.Config.set_tbl_rows(150)
correlations_df
correlations_df.write_excel(workbook="correlations.xlsx")

In [35]:
# pygwalk can be used as easy tool for quick data inspection 
pyg.walk(position_test_df)

Box(children=(HTML(value='\n<div id="ifr-pyg-0006327a8530180brivRwgCQmUPaysIn" style="height: auto">\n    <hea…

## Building the final data frame
For further analysis I wanted to incorporate the data from the ce dataframe into the cleaned combined_json_df

In [72]:
combined_json_df = corr_df
#combined_json_df.shape

ce_df = ce
#ce_df.shape

In [73]:
combined_json_df.head()

Count Sprint OTIP 1,Count Sprint TIP 2,date,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Minutes TIP 1,season_name,Distance,Count Sprint TIP 1,Distance OTIP,Minutes TIP 2,match_name,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Minutes OTIP 2,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Minutes 2,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,Minutes TIP,competition_edition_id,Count Medium Acceleration OTIP 1,player_name,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,group,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Minutes OTIP,player_birthdate,short_name,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,team_name,quality_check,Sprinting Distance 2,Minutes,competition_id,Count HSR TIP 2,Distance 2,PSV-99,Count HSR TIP,Minutes 1,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,position,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Minutes OTIP 1,team,Count Medium Deceleration OTIP 1
i64,i64,str,i64,i64,i64,i64,f64,str,i64,i64,i64,f64,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,i64,f64,i64,i64,i64,f64,i64,f64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,f64,str,i64
1,1,"""2021-07-25""",1,52,17,3,13.13,"""2021/2022""",13026,1,5727,11.08,"""KV Mechelen v Royal Antwerp FC""",8,48,18,30,2,2,17.3,662,1558,33,44999,5,3,22,49.48,3,1874,2744,4,148,26,42,225,24.21,249,24,"""Pieter Gerkens""",0,443,139,6810,11,2,462,8,369,0,"""Midfield""",34,2415,1,4,842,0,0,17,0,3,1504,1354,13,1,35.57,"""1995-02-17""","""P. Gerkens""",59,260,218,17,19,1061,121,27,23,45,2,10,92,12,8701,358,8,11,3432,292,2983,100,89,"""Royal Antwerp FC""",true,56,96.63,16,6,6216,25.7,19,47.15,9,3,34,47,"""AM""",1,661,0,17,745,387,47,18.27,"""Royal Antwerp FC""",17
1,3,"""2021-07-24""",0,36,20,1,13.2,"""2021/2022""",9969,1,3534,12.32,"""Oud-Heverlee Leuven v SV Zulte…",9,41,0,136,5,1,15.07,224,1775,20,45002,8,3,25,50.07,3,1660,1739,1,202,19,31,104,25.52,249,15,"""Xavier Mercier""",1,167,81,4842,8,1,27,8,267,2,"""Midfield""",25,1101,0,5,178,1,0,10,1,2,402,489,9,1,29.59,"""1989-07-25""","""X. Mercier""",39,265,63,14,119,612,184,24,17,24,4,14,48,11,10932,275,5,17,3435,343,1795,77,77,"""Oud-Heverlee Leuven""",true,154,96.19,16,11,5127,28.3,20,46.12,11,3,17,46,"""AM""",2,610,1,7,476,201,32,14.52,"""Oud-Heverlee Leuven""",11
1,0,"""2021-07-24""",1,39,33,2,14.52,"""2021/2022""",8709,2,2876,7.93,"""Oud

In [74]:
#combined_json_df.select("competition_id").unique()

In [75]:
#dropping ce_df columns that are already present in combined_json_df so there is no overlap
ce_df = ce_df.drop(["season_id","season_name", "competition_id"])

In [76]:
ce_df.head()

name,id,season_start_year,season_end_year,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,2014,2014,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2015,2016,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,2016,2016,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,2016,2017,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,2017,2018,"""ENG""","""Premier League""","""male""","""adult"""


In [77]:
# renaming ce_df columns so they don't cause confusion when merging with the combined_json_df
ce_df = ce_df.rename({
    "name":"full_comp_name",
    "id":"competition_edition_id",
})

In [78]:
ce_df.head()

full_comp_name,competition_edition_id,season_start_year,season_end_year,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,2014,2014,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2015,2016,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,2016,2016,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,2016,2017,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,2017,2018,"""ENG""","""Premier League""","""male""","""adult"""


In [79]:
#combined_json_df.select("competition_id").unique()

In [80]:
#bundeled_df

In [81]:
# combined_json_df (competition_edition_id) and ce_df (id) match 
# intialize a list to save all dictionaries + and then concat them into a dataframe (as before)
# joining dataframes on the common competition edition id
#creating the final / work df 

work_df = combined_json_df.join(ce_df, on="competition_edition_id")
work_df.shape

(449537, 124)

In [84]:
work_df.head()

Count Sprint OTIP 1,Count Sprint TIP 2,date,Count High Deceleration OTIP 2,Count Medium Acceleration 1,Count Medium Deceleration TIP,Count High Deceleration OTIP 1,Minutes TIP 1,season_name,Distance,Count Sprint TIP 1,Distance OTIP,Minutes TIP 2,match_name,Count Medium Deceleration TIP 1,Count Medium Acceleration 2,Sprinting Distance OTIP 2,Sprinting Distance TIP,Count Sprint 2,Count High Deceleration TIP 1,Minutes OTIP 2,Running Distance OTIP 2,Distance TIP 2,Count HSR 1,match_id,Count Sprint,Count High Acceleration 1,Count Medium Acceleration TIP,Minutes 2,Count Sprint 1,Distance TIP 1,Distance OTIP 2,Count High Deceleration OTIP,Sprinting Distance,Count HSR 2,Count Medium Deceleration 1,HSR Distance OTIP 1,Minutes TIP,competition_edition_id,Count Medium Acceleration OTIP 1,player_name,Count High Acceleration TIP 2,HSR Distance OTIP,HSR Distance TIP 1,Distance 1,Count High Deceleration,Count Sprint OTIP,team_id,season_id,Running Distance TIP 1,Count High Acceleration TIP,group,Count Medium Deceleration OTIP,Running Distance,Count Sprint OTIP 2,Count High Acceleration,Running Distance OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration OTIP 1,Count HSR OTIP 1,Count High Acceleration TIP 1,Count High Deceleration TIP,Running Distance OTIP,Running Distance 1,Count HSR TIP 1,Count High Deceleration TIP 2,Minutes OTIP,player_birthdate,short_name,Count HSR,HSR Distance TIP,HSR Distance OTIP 2,Count Medium Deceleration OTIP 2,Sprinting Distance TIP 2,Running Distance 2,HSR Distance TIP 2,Sprinting Distance OTIP 1,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Count Sprint TIP,Count Medium Acceleration TIP 2,Sprinting Distance 1,Count Medium Acceleration TIP 1,player_id,HSR Distance 2,Count High Deceleration 1,Sprinting Distance TIP 1,Distance TIP,Running Distance TIP 2,Distance OTIP 1,Count Medium Acceleration,Count Medium Deceleration,team_name,quality_check,Sprinting Distance 2,Minutes,competition_id,Count HSR TIP 2,Distance 2,PSV-99,Count HSR TIP,Minutes 1,Count Medium Deceleration TIP 2,Count High Deceleration 2,Count HSR OTIP,Count Medium Deceleration 2,position,Count High Acceleration 2,Running Distance TIP,Count High Acceleration OTIP,Count HSR OTIP 2,HSR Distance,HSR Distance 1,Count Medium Acceleration OTIP,Minutes OTIP 1,team,Count Medium Deceleration OTIP 1,full_comp_name,season_start_year,season_end_year,competition_area,competition_name,competition_gender,competition_age_group
i64,i64,str,i64,i64,i64,i64,f64,str,i64,i64,i64,f64,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,i64,f64,i64,i64,i64,f64,i64,f64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,f64,str,i64,str,i64,i64,str,str,str,str
1,1,"""2021-07-25""",1,52,17,3,13.13,"""2021/2022""",13026,1,5727,11.08,"""KV Mechelen v Royal Antwerp FC""",8,48,18,30,2,2,17.3,662,1558,33,44999,5,3,22,49.48,3,1874,2744,4,148,26,42,225,24.21,249,24,"""Pieter Gerkens""",0,443,139,6810,11,2,462,8,369,0,"""Midfield""",34,2415,1,4,842,0,0,17,0,3,1504,1354,13,1,35.57,"""1995-02-17""","""P. Gerkens""",59,260,218,17,19,1061,121,27,23,45,2,10,92,12,8701,358,8,11,3432,292,2983,100,89,"""Royal Antwerp FC""",true,56,96.63,16,6,6216,25.7,19,47.15,9,3,34,47,"""AM""",1,661,0,17,745,387,47,18.27,"""Royal Antwerp FC""",17,"""BEL - Pro League - 2021/2022""",2021,2022,"""BEL""","""Pro League""","""male""","""adult"""
1,3,"""2021-07-24""",0,36,20,1,13.2,"""2021/2022""",9969,1,3534,12.32,"""Oud-Heverlee Leuven v SV Zulte…",9,41,0,136,5,1,15.07,224,1775,20,45002,8,3,25,50.07,3,1660,1739,1,202,19,31,104,25.52,249,15,"""Xavier Mercier""",1,167,81,4842,8,1,27,8,267,2,"""Midfield""",25,1101,0,5,178,1,0,10,1,2,402,489,9,1,29.59,"""1989-07-25""","""X. Mercier""",39,265,63,14,119,612,184,24,17,24,4,14,48,11,10932,

In [85]:
# trying the same with left join to see if it gives different results
'''
work_df2 = combined_json_df.join(ce_df, on="competition_edition_id", how = 'left')
work_df2.shape'''

'\nwork_df2 = combined_json_df.join(ce_df, on="competition_edition_id", how = \'left\')\nwork_df2.shape'

In [86]:
# drop uninformative columns

In [96]:
work_df = work_df.select(sorted(work_df.columns))

In [97]:
work_df.head()

Count HSR,Count HSR 1,Count HSR 2,Count HSR OTIP,Count HSR OTIP 1,Count HSR OTIP 2,Count HSR TIP,Count HSR TIP 1,Count HSR TIP 2,Count High Acceleration,Count High Acceleration 1,Count High Acceleration 2,Count High Acceleration OTIP,Count High Acceleration OTIP 1,Count High Acceleration OTIP 2,Count High Acceleration TIP,Count High Acceleration TIP 1,Count High Acceleration TIP 2,Count High Deceleration,Count High Deceleration 1,Count High Deceleration 2,Count High Deceleration OTIP,Count High Deceleration OTIP 1,Count High Deceleration OTIP 2,Count High Deceleration TIP,Count High Deceleration TIP 1,Count High Deceleration TIP 2,Count Medium Acceleration,Count Medium Acceleration 1,Count Medium Acceleration 2,Count Medium Acceleration OTIP,Count Medium Acceleration OTIP 1,Count Medium Acceleration OTIP 2,Count Medium Acceleration TIP,Count Medium Acceleration TIP 1,Count Medium Acceleration TIP 2,Count Medium Deceleration,Count Medium Deceleration 1,Count Medium Deceleration 2,Count Medium Deceleration OTIP,Count Medium Deceleration OTIP 1,Count Medium Deceleration OTIP 2,Count Medium Deceleration TIP,Count Medium Deceleration TIP 1,Count Medium Deceleration TIP 2,Count Sprint,Count Sprint 1,Count Sprint 2,Count Sprint OTIP,Count Sprint OTIP 1,Count Sprint OTIP 2,Count Sprint TIP,Count Sprint TIP 1,Count Sprint TIP 2,Distance,Distance 1,Distance 2,Distance OTIP,Distance OTIP 1,Distance OTIP 2,Distance TIP,Distance TIP 1,Distance TIP 2,HSR Distance,HSR Distance 1,HSR Distance 2,HSR Distance OTIP,HSR Distance OTIP 1,HSR Distance OTIP 2,HSR Distance TIP,HSR Distance TIP 1,HSR Distance TIP 2,Minutes,Minutes 1,Minutes 2,Minutes OTIP,Minutes OTIP 1,Minutes OTIP 2,Minutes TIP,Minutes TIP 1,Minutes TIP 2,PSV-99,Running Distance,Running Distance 1,Running Distance 2,Running Distance OTIP,Running Distance OTIP 1,Running Distance OTIP 2,Running Distance TIP,Running Distance TIP 1,Running Distance TIP 2,Sprinting Distance,Sprinting Distance 1,Sprinting Distance 2,Sprinting Distance OTIP,Sprinting Distance OTIP 1,Sprinting Distance OTIP 2,Sprinting Distance TIP,Sprinting Distance TIP 1,Sprinting Distance TIP 2,competition_age_group,competition_area,competition_edition_id,competition_gender,competition_id,competition_name,date,full_comp_name,group,match_id,match_name,player_birthdate,player_id,player_name,position,quality_check,season_end_year,season_id,season_name,season_start_year,short_name,team,team_id,team_name
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,str,i64,str,str,str,str,i64,str,str,i64,str,str,bool,i64,i64,str,i64,str,str,i64,str
59,33,26,34,17,17,19,13,6,4,3,1,0,0,0,0,0,0,11,8,3,4,3,1,3,2,1,100,52,48,47,24,23,22,12,10,89,42,47,34,17,17,17,8,9,5,3,2,2,1,1,2,1,1,13026,6810,6216,5727,2983,2744,3432,1874,1558,745,387,358,443,225,218,260,139,121,96.63,47.15,49.48,35.57,18.27,17.3,24.21,13.13,11.08,25.7,2415,1354,1061,1504,842,662,661,369,292,148,92,56,45,27,18,30,11,19,"""adult""","""BEL""",249,"""male""",16,"""Pro League""","""2021-07-25""","""BEL - Pro League - 2021/2022""","""Midfield""",44999,"""KV Mechelen v Royal Antwerp FC""","""1995-02-17""",8701,"""Pieter Gerkens""","""AM""",true,2022,8,"""2021/2022""",2021,"""P. Gerkens""","""Royal Antwerp FC""",462,"""Royal Antwerp FC"""
39,20,19,17,10,7,20,9,11,5,3,2,1,0,1,2,1,1,8,5,3,1,1,0,2,1,1,77,36,41,32,15,17,25,11,14,77,31,46,25,11,14,20,9,11,8,3,5,1,1,0,4,1,3,9969,4842,5127,3534,1795,1739,3435,1660,1775,476,201,275,167,104,63,265,81,184,96.19,46.12,50.07,29.59,14.52,15.07,25.52,13.2,12.32,28.3,1101,489,612,402,178,224,610,267,343,202,48,154,24,24,0,136,17,119,"""adult""","""BEL""",249,"""male""",16,"""Pro League""","

In [98]:
work_df.columns

['Count HSR',
 'Count HSR 1',
 'Count HSR 2',
 'Count HSR OTIP',
 'Count HSR OTIP 1',
 'Count HSR OTIP 2',
 'Count HSR TIP',
 'Count HSR TIP 1',
 'Count HSR TIP 2',
 'Count High Acceleration',
 'Count High Acceleration 1',
 'Count High Acceleration 2',
 'Count High Acceleration OTIP',
 'Count High Acceleration OTIP 1',
 'Count High Acceleration OTIP 2',
 'Count High Acceleration TIP',
 'Count High Acceleration TIP 1',
 'Count High Acceleration TIP 2',
 'Count High Deceleration',
 'Count High Deceleration 1',
 'Count High Deceleration 2',
 'Count High Deceleration OTIP',
 'Count High Deceleration OTIP 1',
 'Count High Deceleration OTIP 2',
 'Count High Deceleration TIP',
 'Count High Deceleration TIP 1',
 'Count High Deceleration TIP 2',
 'Count Medium Acceleration',
 'Count Medium Acceleration 1',
 'Count Medium Acceleration 2',
 'Count Medium Acceleration OTIP',
 'Count Medium Acceleration OTIP 1',
 'Count Medium Acceleration OTIP 2',
 'Count Medium Acceleration TIP',
 'Count Medium A

In [99]:
#creating the final / work df 
work_df.write_csv('data_summary.csv')

: 